In [ ]:
!unzip '/content/Customer-classification.zip'

Archive:  /content/Customer-classification.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Yes
  inflating: sample_submission.csv   
replace existing_base_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: existing_base_test.csv  
replace existing_base_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: existing_base_train.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier # Standard ML Model
from sklearn.linear_model import Perceptron #Neural Network Model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #Model Evaluation

In [ ]:
train = pd.read_csv('/content/existing_base_train.csv')
test = pd.read_csv('/content/existing_base_test.csv')

In [ ]:
train.isna().sum()

REF_NO                             0
children                           0
age_band                           0
status                             0
occupation                         0
occupation_partner                 0
home_status                        0
family_income                      0
self_employed                      0
self_employed_partner              0
year_last_moved                    0
TVarea                             0
post_code                          0
post_area                          0
Average_Credit_Card_Transaction    0
Balance_Transfer                   0
Term_Deposit                       0
Life_Insurance                     0
Medical_Insurance                  0
Average_A/C_Balance                0
Personal_Loan                      0
Investment_in_Mutual_Fund          0
Investment_Tax_Saving_Bond         0
Home_Loan                          0
Online_Purchase_Amount             0
gender                             0
region                             0
I

In [ ]:
train.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,gender,region,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid
0,5466,2,31-35,Partner,Professional,Professional,Own Home,">=35,000",No,No,...,8.98,55.44,7.68,Female,North West,151.55,81.79,136.02,360.37,2
1,9091,Zero,45-50,Partner,Secretarial/Admin,Professional,Own Home,">=35,000",No,No,...,0.00,0.00,18.99,Female,North West,44.28,13.91,29.23,89.22,2
2,9744,1,36-40,Partner,Manual Worker,Manual Worker,Rent Privately,"<22,500, >=20,000",Yes,Yes,...,0.00,0.00,0.00,Female,South West,8.58,1.75,4.82,14.50,2
3,10700,2,31-35,Partner,Manual Worker,Manual Worker,Own Home,"<25,000, >=22,500",No,No,...,0.00,0.00,0.00,Female,Scotland,15.00,0.00,5.00,68.98,2
4,1987,Zero,55-60,Partner,Housewife,Professional,Own Home,">=35,000",No,No,...,0.00,0.00,0.00,Female,Unknown,0.00,1.66,1.66,1.88,2


In [ ]:
# Function to map age range to specific age (midpoint)
def map_age_range(age_range):
    if '-' in age_range:
        lower, upper = age_range.split('-')
        specific_age = (int(lower) + int(upper)) // 2
        return specific_age
    return age_range

In [ ]:
# Define the replacement value for "4+"
replacement_value = 5
# Function to map "4+" to the replacement value
def replace_four_plus(value):
    if value == '4+':
        return replacement_value
    return value

In [ ]:
# Define the replacement value for "4+"
replacement_value = 72
# Function to map "4+" to the replacement value
def replace_seventy_plus(value):
    if value == '71+':
        return replacement_value
    return value

In [ ]:
train['age_band']= train['age_band'].apply(map_age_range)

In [ ]:
train['children']= train['children'].replace('Zero', 0)

In [ ]:
train['age_band'] = train['age_band'].apply(replace_four_plus)

In [ ]:
train['children'] = train['children'].apply(replace_four_plus)

In [ ]:
train['age_band'] = train['age_band'].apply(replace_seventy_plus)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REF_NO                           2031 non-null   int64  
 1   children                         2031 non-null   object 
 2   age_band                         2031 non-null   object 
 3   status                           2031 non-null   object 
 4   occupation                       2031 non-null   object 
 5   occupation_partner               2031 non-null   object 
 6   home_status                      2031 non-null   object 
 7   family_income                    2031 non-null   object 
 8   self_employed                    2031 non-null   object 
 9   self_employed_partner            2031 non-null   object 
 10  year_last_moved                  2031 non-null   int64  
 11  TVarea                           2031 non-null   object 
 12  post_code           

In [ ]:
# Convert columns to float type
train['status'] = train['status'].astype(str)
#train['occupation_partner'] = train['occupation_partner'].astype(str)
train['occupation'] = train['occupation'].astype(str)
train['region'] = train['region'].astype(str)

train['self_employed_partner'] = train['self_employed_partner'].astype(str)
train['self_employed'] = train['self_employed'].astype(str)
train['home_status'] = train['home_status'].astype(str)

In [ ]:
train['gender'] = train['gender'].astype(object)

In [ ]:
# Remove rows where any of the specified columns contain 'unknown'
train = train[(train['region'] != 'Unknown') & (train['TVarea'] != 'Unknown') & (train['gender'] != 'Unknown')  & (train['status'] != 'Unknown')]
train = train[(train['occupation'] != 'Unknown')& (train['age_band'] != 'Unknown') & (train['family_income'] != 'Unknown') ]

In [ ]:
#train = train .drop(columns=['occupation_partner'])
train = train .drop(columns=['status'])
train = train .drop(columns=['post_area'])
train = train .drop(columns=['self_employed_partner'])
train = train .drop(columns=['TVarea'])

In [ ]:
# Function to clean and convert values
def clean_and_convert(value):
    try:
        # Remove non-numeric characters and convert to float
        #cleaned_value = float(str(value).replace('>=', '').replace(',', ''))
        cleaned_value = int(train['family_income'].split(',')[1].strip().replace('>=', ''))
        return cleaned_value
    except ValueError:
        return None  # Handle cases where conversion fails

In [ ]:
# Function to transform Income based on the provided conditions
def transform_income(income):
    if ',' in income:  # Check if the income value is a range
        # Split the string by commas and process each part
        parts = income.split(',')
        for part in parts:
            # Remove any non-numeric characters like '<', '>', and whitespace
            numeric_part = ''.join(filter(str.isdigit, part))
            if numeric_part:
                return int(numeric_part+'000')  # Convert the numeric part to integer
    else:
        # If there are no commas, just convert the whole string to integer
        numeric_part = ''.join(filter(str.isdigit, income))
        return int(numeric_part+'000')

In [ ]:
train['family_income'] = train['family_income'].apply(transform_income)

In [ ]:
import pandas as pd

# Define the data
data = {
    'Family': ['A', 'B', 'C'],
    'Income': ['35', '<35, >=20', '67']
}

# Create the DataFrame
df = pd.DataFrame(data)

# Function to transform Income based on the provided conditions
def transform_income(income):
    if ',' in income:  # Check if the income value is a range
        min_income = int(income.split(',')[1].strip().replace('>=', ''))
        return min_income
    else:
        return int(income)  # Convert single income values to integer

# Apply the function to transform Income column
df['Income'] = df['Income'].apply(transform_income)

# Print the DataFrame
print(df)

  Family  Income
0      A      35
1      B      20
2      C      67


In [ ]:
train = train .drop(columns=['status'])

In [ ]:
train = train .drop(columns=['occupation_partner'])

In [ ]:
train = train .drop(columns=['self_employed'])
train = train .drop(columns=['self_employed_partner'])

In [ ]:
train = train .drop(columns=['TVarea'])
train = train .drop(columns=['post_code'])
train = train .drop(columns=['post_area'])


In [ ]:
train

,REF_NO,children,age_band,family_income,year_last_moved,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,Life_Insurance,Medical_Insurance,...,Investment_in_Mutual_Fund,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,gender,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid
0,5466,2,33,35000,1981,26.98,29.99,312.25,299.79,88.72,...,134.35,8.98,55.44,7.68,Female,151.55,81.79,136.02,360.37,2
1,9091,0,47,35000,1997,35.98,74.48,0.00,99.96,10.99,...,0.00,0.00,0.00,18.99,Female,44.28,13.91,29.23,89.22,2
2,9744,1,38,22000,1996,0.00,24.46,0.00,18.44,0.00,...,10.46,0.00,0.00,0.00,Female,8.58,1.75,4.82,14.50,2
3,10700,2,33,25000,1990,44.99,0.00,0.00,0.00,29.99,...,0.00,0.00,0.00,0.00,Female,15.00,0.00,5.00,68.98,2
5,3309,0,47,35000,1984,9.49,0.01,0.00,0.51,55.89,...,0.00,0.00,0.00,0.00,Female,13.18,4.83,14.23,33.62,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,6516,3,38,20000,1981,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,Male,0.00,0.00,0.00,15.23,2
8120,5897,0,63,8000,1960,0.00,0.00,9.49,0.00,0.00,...,46.76,0.00,0.00,0.00,Female,1.90,8.29,8.29,68.42,2
8121,6130,1,43,8000,1987,0.00,107.42,0.00,23.42,23.99,...,101.37,1.00,3.72,0.00,Female,30.97,24.17,31.46,106.06,2
8122,980,0,63,4000,1985,0.00,59.48,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,Female,11.90,0.00,0.00,-9.19,2


In [ ]:
train = train .drop(columns=['region'])

In [ ]:
train = train .drop(columns=['home_status'])

In [ ]:
train = train .drop(columns=['occupation'])

In [ ]:
# Filter rows where 'Experience' column contains '4+'
train4 = train[train.applymap(lambda x: '4+' in str(x)).any(axis=1)]
train71 = train[train.applymap(lambda x: '71+' in str(x)).any(axis=1)]
trainUnknown = train[train.applymap(lambda x: 'Unknown' in str(x)).any(axis=1)]
trainreg = train[train.applymap(lambda x: 'North West' in str(x)).any(axis=1)]

In [ ]:
print(trainreg)

Empty DataFrame
Columns: [REF_NO, children, age_band, status, occupation_partner, family_income, self_employed, self_employed_partner, year_last_moved, TVarea, post_code, post_area, Average_Credit_Card_Transaction, Balance_Transfer, Term_Deposit, Life_Insurance, Medical_Insurance, Average_A/C_Balance, Personal_Loan, Investment_in_Mutual_Fund, Investment_Tax_Saving_Bond, Home_Loan, Online_Purchase_Amount, gender, Investment_in_Commudity, Investment_in_Equity, Investment_in_Derivative, Portfolio_Balance, Revenue_Grid]
Index: []

[0 rows x 29 columns]


In [ ]:
train.describe()

,REF_NO,family_income,year_last_moved,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,Life_Insurance,Medical_Insurance,Average_A/C_Balance,Personal_Loan,Investment_in_Mutual_Fund,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid
count,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000,6729.000000
mean,5797.557289,23685.688810,1980.318621,23.573662,45.954359,27.502364,65.279960,18.834475,31.848331,26.157178,41.871236,5.978150,4.425426,19.368948,36.229144,21.608984,31.662396,89.751434,1.894041
std,3323.899784,9324.591844,94.425131,52.154857,80.484884,54.773867,96.799462,32.327753,45.361481,90.299569,65.530180,12.585724,10.055970,93.282313,43.299845,33.156918,40.684381,111.216137,0.307808
min,2.000000,4000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-76.350000,1.000000
25%,2919.000000,15000.000000,1978.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.200000,4.670000,8.660000,26.420000,2.000000
50%,5804.000000,25000.000000,1988.000000,0.000000,17.490000,0.000000,31.470000,0.000000,14.980000,0.000000,23.480000,0.000000,0.000000,0.000000,23.590000,12.830000,21.230000,65.470000,2.000000
75%,8697.000000,35000.000000,1994.000000,23.480000,64.970000,34.980000,92.910000,26.480000,45.370000,20.980000,59.450000,5.470000,4.490000,7.480000,49.680000,28.050000,42.550000,124.670000,2.000000
max,11514.000000,35000.000000,1999.000000,662.260000,2951.760000,784.820000,2930.410000,591.040000,626.240000,4905.930000,2561.270000,138.560000,162.350000,4306.420000,1231.090000,1279.100000,1771.160000,4283.560000,2.000000


In [ ]:
# Perform one-hot encoding with handling of all categories
train = pd.get_dummies(train, columns=['gender'], dummy_na=1)
# Convert encoded columns to integers explicitly
train = train.astype(int)

Hot encoding for Gender

In [ ]:
train.head()

,REF_NO,children,age_band,family_income,year_last_moved,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,Life_Insurance,Medical_Insurance,...,Home_Loan,Online_Purchase_Amount,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid,gender_Female,gender_Male,gender_nan
0,5466,2,33,35000,1981,26,29,312,299,88,...,55,7,151,81,136,360,2,1,0,0
1,9091,0,47,35000,1997,35,74,0,99,10,...,0,18,44,13,29,89,2,1,0,0
2,9744,1,38,22000,1996,0,24,0,18,0,...,0,0,8,1,4,14,2,1,0,0
3,10700,2,33,25000,1990,44,0,0,0,29,...,0,0,15,0,5,68,2,1,0,0
5,3309,0,47,35000,1984,9,0,0,0,55,...,0,0,13,4,14,33,2,1,0,0


In [ ]:
train.shape

(6729, 27)

In [ ]:
test.shape

(2031, 31)

In [ ]:
corr = train.corr()

Split the data for Training & Testing

In [ ]:
x= train.drop(['Revenue_Grid'],axis=1)
y= train['Revenue_Grid']

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5046, 23)
(1683, 23)
(5046,)
(1683,)


In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
# y_hat_train = rf.predict(x_train)
y_hat_test = rf.predict(x_test)

In [ ]:
error = f1_score(y_test, y_hat_test)
error

0.8036253776435046

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error

In [ ]:
mymodel = LogisticRegression()

In [ ]:
mymodel.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
LogisticRegression()

LogisticRegression()

In [ ]:
y_hat_train = mymodel.predict(x_train)
y_hat_test = mymodel.predict(x_test)

In [ ]:
error = f1_score(y_test, y_hat_test)

In [ ]:
error

0.7004219409282699

In [ ]:
errormean = mean_absolute_error(y_train, y_hat_train)
errormean

0.05721716514954486

In [ ]:
 def model_eval(actual, predicted):
    print("Accuracy_Score", round(accuracy_score(actual,predicted),2))
    print(confusion_matrix(actual,predicted))
    print(classification_report(actual,predicted))

In [ ]:
model_eval(y_train , y_hat_train)

Accuracy_Score 0.94
[[ 332  240]
 [  68 4743]]
              precision    recall  f1-score   support

           1       0.83      0.58      0.68       572
           2       0.95      0.99      0.97      4811

    accuracy                           0.94      5383
   macro avg       0.89      0.78      0.83      5383
weighted avg       0.94      0.94      0.94      5383

